In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# 실습용 파일 다운로드

from urllib import request

url = "https://raw.githubusercontent.com/jehyunlee/2021_KSES_ML/main/penguins_pega.csv"
savename = "data.csv"

request.urlretrieve(url, savename)

('data.csv', <http.client.HTTPMessage at 0x7ffba7b75510>)

In [6]:
# 실습용 파일 열기

df = pd.read_csv("data.csv")
df.head()

,Unnamed: 0,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,id
0,0,Gentoo,Biscoe,46.3,15.8,215.0,5050.0,Male,1414
1,1,Adelie,Biscoe,39.6,20.7,191.0,3900.0,Female,1693
2,2,Gentoo,Biscoe,46.5,14.5,213.0,4400.0,Female,1954
3,3,Adelie,Dream,39.7,17.9,193.0,4250.0,Male,1056
4,4,Adelie,Dream,36.3,19.5,190.0,3800.0,Male,1160
